# Part I. ETL Pipeline for Pre-Processing the Files

## BELOW IS THE CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [79]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [80]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [81]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [82]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion. 

## The <font color=red>event_datafile_new.csv</font>, created above contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [83]:
# This makes a connection to a Cassandra instance to local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [84]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [85]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Tables are created below by modeling the data

## Below are the queries considered for modeling data and creating the tables

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# First Table

In [87]:
## Creating Table 1 - song_details

query = "CREATE TABLE IF NOT EXISTS song_details "
query = query + "(artist text, \
                    song text, \
                    length float, \
                    sessionId int, \
                    itemInSession int, \
                    PRIMARY KEY (sessionId, itemInSession)\
                )"
try:
    session.execute(query)
except Exception as e:
    print(e)
    print("Error in creating song_details table")
                    

In [88]:
# Inserting data into song_details table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_details(artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Below is the SELECT to verify that the data have been inserted into each table

In [89]:
## Select query
query = "select artist,song,length from song_details WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df=pd.DataFrame()

for i,row in enumerate(rows,1):
    df = df.append(pd.DataFrame({'Artist':[row.artist],'Song':[row.song],'Length':[row.length]},columns=['Artist','Song','Length'],index=[i]))
df

,Artist,Song,Length
1,Faithless,Music Matters (Mark Knight Dub),495.307312


# Second Table

In [90]:
## Creating Table 2 - user_session_songs
query = "CREATE TABLE IF NOT EXISTS user_session_songs "
query = query + "(artist text, \
                    song text, \
                    firstName text, \
                    lastName text, \
                    userId int, \
                    sessionId int, \
                    itemInSession int, \
                    PRIMARY KEY((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    print("Error in creating user_session_songs")                                      

In [91]:
# Inserting data into user_session_songs table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_songs(artist, song, firstName,lastName,userId,sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Below is the SELECT to verify that the data have been inserted into each table

In [92]:
## Select query
query = "select artist,song,firstName,lastName from user_session_songs WHERE sessionId = 182 and userId=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df=pd.DataFrame()

for i,row in enumerate(rows,1):
    df = df.append(pd.DataFrame({'Artist':[row.artist],'Song':[row.song],'FirstName':[row.firstname],'LastName':[row.lastname]},\
                                columns=['Artist','Song','FirstName','LastName'],index=[i]))
df

,Artist,Song,FirstName,LastName
1,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Sebastien Tellier,Kilometer,Sylvie,Cruz
4,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [93]:
# Third Table

In [94]:
## Creating Table 3 - song_list_by_user
query = "CREATE TABLE IF NOT EXISTS song_list_by_user "
query = query + "(firstName text, lastName text, song text, PRIMARY KEY(song,firstName,lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    print("Error in creating song_list_by_user")                                      
                    

In [95]:
# Inserting data into song_list table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_list_by_user(firstName,lastName,song)"
        query = query + "VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1],line[4],line[9]))

#### Below is the SELECT to verify that the data have been inserted into each table

In [98]:
## Select query
query = "select firstname,lastname from song_list_by_user where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df=pd.DataFrame()

for i,row in enumerate(rows,1):
    df = df.append(pd.DataFrame({'FirstName':[row.firstname],'LastName':[row.lastname]},\
                                columns=['FirstName','LastName'],index=[i]))
df    

,FirstName,LastName
1,Jacqueline,Lynch
2,Sara,Johnson
3,Tegan,Levine


### Dropping the tables before closing out the sessions

In [99]:
query = "DROP TABLE IF EXISTS song_details"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    print("Error in dropping table song_details")

query = "DROP TABLE IF EXISTS user_songs"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    print("Error in dropping table user_session_songs")
    
query = "DROP TABLE IF EXISTS song_list_by_user"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    print("Error in dropping table song_list_by_user")    

### Closing the session and cluster connection¶

In [100]:
session.shutdown()
cluster.shutdown()